# Lesson 02: geospatial datasets

In this lesson we will use a typical use case in glacier research as a excuse to learn something about simple geospatial checks.

The main question we are trying to address is: how do we know which of the > 200'000 glaciers worldwide are located in the Rhone basin? And in the Indus basin?

Let's start by importing the packages. We have a new package this week!

<div class="alert alert-success">
    This workshop and the next require the installation of new python packages. This needs to be done once on the machine you are running this notebook. You can head to [the install instruction]() page to find the instructions.
</div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from cartopy import crs as ccrs
import geopandas as gpd  # This is new!
import shapely.geometry as shpg  # This is new as well

We introduce two new packages:

- [**GeoPandas**](https://geopandas.org/en/stable/) extends Pandas to handle spatial data, making it easier to work with geographic  datasets such as river basins, and much, much more.
- [**Shapely**](https://shapely.readthedocs.io/en/stable/) provides geometric objects and operations, which geopandas uses under the hood to manipulate spatial features. We'll cross that bridge when it comes.

## Global glacier locations and other statistics

Head to the [data download page]() and download the two files:
- `glacier_basins.zip`: I put this file in a new `glaciers` folder I created in the `data` folder, and extracted it (the zip extraction is important!)
- `rgi60_stats.csv`: I also put this file in the `glaciers` folder

The `rgi60_stats.csv` is a csv file - we know how to deal with these files now:

In [ ]:
rgi_df = pd.read_csv('../data/glaciers/rgi60_stats.csv', index_col=0)

Explore the dataframe, its structure, length, etc.

In [ ]:
# Your answer here

This table was generated by me based on the [Randolph Glacier Inventory](https://www.glims.org/RGI/) version 6. 

Here are a few explanations for the most relevant columns:

- **RGIId**: The unique identifier for each glacier in the Randolph Glacier Inventory (RGI).  
- **BgnDate**: The date of the satellite image used to delineate the glacier, in YYYYMMDD format.  
- **CenLon, CenLat**: The central longitude and latitude of the glacier, representing its approximate centroid.  
- **O1Region, O2Region**: The primary and secondary region codes according to the RGI classification. These help categorize glaciers by geographic location.  
- **Area**: The glacier’s total surface area in square kilometers (km²).  
- **Zmin, Zmax, Zmed**: The minimum, maximum, and median elevation of the glacier in meters above sea level (m a.s.l.).  
- **Slope**: The average surface slope of the glacier, measured in degrees.  
- **Aspect**: The dominant aspect (orientation) of the glacier, measured in degrees (0° = North, 90° = East, 180° = South, 270° = West).  
- **Name**: The given name of the glacier, if available. Many glaciers remain unnamed and are recorded as `NaN`.

This dataset provides a static snapshot of glaciers at the time of their delineation, meaning it does not track their changes over time. The RGI's primary goal is to provide glacier **outlines** - here I scrapped them out of the file and only provide some statistics.

Here are a few questions to get you started:
- how many glaciers are there in the RGI?
- what is their total area?
- what is the maximum glacier elevation?
- can you plot a histogram of the `Aspect` of the glaciers? What is the dominant direction of glaciers? Was it predictable?
- optional: select all glaciers in the northern hemisphere, and repeat the histogram
- optional: now group the glaciers per `O1Region` (hint: `groupby()`), and compute the total area per RGI region

In [ ]:
# Your answer here

The RGI glaciers are organised in main regions as shown on this map:


<img src="https://raw.githubusercontent.com/GLIMS-RGI/rgi_user_guide/main/docs/img/global_stats/global_map.jpeg" alt="Global Glacier Map" width="700">

Good! There is a lot more we could explore with this dataset, but we have to get going with more pressing matters. We'd like to use this dataset to select glacier model projection data for the glaciers of our choice. In the next section, we'll try to find out which of all these glaciers is located in the Rhone basin.

For this the `CenLon` and `CenLat` columns might be useful, but there is so much we can do with a table. I can start by plotting the glacier locations on a scatter plot, but this is not really "geospatial" yet:

In [ ]:
rgi_df.plot.scatter(x='CenLon', y='CenLat', s=0.1);

## River basins shapefile

Let's open the second dataset you downloaded. You should have a `glacier_basins` folder containing five files. In a way, these are all parts of the same dataset, which is referenced by the main file: **`glacier_basins.shp`**.

A **Shapefile** is a widely used format for storing vector data in Geographic Information Systems (GIS). Despite the name, a Shapefile is actually a collection of multiple files that work together.
Shapefiles store **vector data**, which represents geographic features using **Points** (e.g., glacier centroids), **Lines** (e.g., river networks), **Polygons** (e.g., glacier outlines and basins).  

Unlike raster data (grids of pixels), vector data allows for precise geographic representation and rich attribute information, and it is structured very differently.

We can now load the `glacier_basins.shp` file using GeoPandas to explore its content:

In [ ]:
gdf = gpd.read_file('../data/glaciers/glacier_basins/glacier_basins.shp')  

`gdf` is called a "geodataframe". You'll find it very convenient, because it ressembles a very normal pandas dataframe in all but one column (`geometry`):

In [ ]:
gdf

In [ ]:
gdf.head()

Operations that work on a dataframe also work on a geodataframe. For example, we can count the number of basins per continent:

In [ ]:
gdf.groupby('CONTINENT').count()[['geometry']]

**Note that this is a subset of all of the word basins. These are all of the world’s large river basins (> 3000 km²) with considerable glacier cover (> 30 km²).** The full dataset can be found at the [Global Runoff Data Centre - Major River Basins](https://grdc.bafg.de/products/basin_layers/major_rivers/).

Using standard pandas methods, we can also compute the total area of glaciers contained in these basins:

In [ ]:
gdf['RGI_AREA'].sum()

*Note: This is much less than the total area of glaciers worldwide you compute above, because many glaciers in the polar regions do not have an assigned basin.*

However, Geopandas differs from pandas when plotting. When asked to plot a geodataframe, geopandas will plot the geometries:

In [ ]:
gdf.plot(ec='k');

Geopandas also comes with a lot of handy ways to plot the data, but we'll learn only one today: assigning colors to attributes of the geometries. Here coloring by basin area:

In [ ]:
f, ax = plt.subplots(figsize=(12, 4))
gdf.plot(ax=ax, column="AREA_CALC", cmap="OrRd", ec='k', legend=True);
plt.title('Basin area');

**Exercise: now plot the basins again but the color indicating glacier area, not basin area.**

In [ ]:
# Your answer here

Bonus: one can also plot categorical data! See by yourself:

In [ ]:
f, ax = plt.subplots(figsize=(12, 4))
gdf.plot(ax=ax, column="OCEAN", categorical=True, cmap="Pastel1", ec='k', 
         legend=True, legend_kwds={"loc": "lower right"});
plt.title('In which ocean does each river flow?'); 

And, finally, one can plot on a map projection with cartopy, very much like the first workshops:

In [ ]:
# Create figure with custom figsize and projection
fig, ax = plt.subplots(figsize=(12, 4), subplot_kw={"projection": ccrs.Robinson()})
gdf.plot(ax=ax, column="OCEAN", categorical=True, cmap="Pastel1", ec='k', 
         legend=True, legend_kwds={"loc": "lower right"},
         transform=ccrs.PlateCarree());
plt.title('In which ocean does each river flow?'); 
ax.coastlines(color='grey'); ax.set_global();

## Finding out in which basin each glacier belongs

So far, we have a dataset of **glaciers** (`rgi_df`) with their geographic coordinates (`CenLon`, `CenLat`), but no spatial information about which river basin they belong to. On the other hand, we have a **GeoDataFrame** (`gdf`) that contains the river basin boundaries as polygons.

To assign each glacier to a basin, we will:
1. **Convert glacier locations into geometries**: We will transform `CenLon` and `CenLat` into Shapely Point objects, creating a **GeoDataFrame** from `rgi_df`.
2. **Use a spatial join**: We will use `geopandas.sjoin()` to determine which basin each glacier falls into.
3. **Store the basin information**: The result will be a new column in `rgi_df` that tells us which basin each glacier belongs to.

This is a common GIS task in climate impact studies, as linking a dataset to another is a very important step for most impact or vulnerability assessment. Let's do it!

In [ ]:
# Step 1: Convert RGI dataframe into a GeoDataFrame with Point geometries
rgi_gdf = gpd.GeoDataFrame(
    rgi_df,
    geometry=gpd.points_from_xy(rgi_df['CenLon'], rgi_df['CenLat']),
    crs="EPSG:4326"  # This is WGS84 - the same coordinate reference system as the basins
)

# Step 2: Perform a spatial join (each glacier gets assigned a basin)
rgi_gdf = gpd.sjoin(rgi_gdf, gdf, how="left", predicate="within")

# Step 3: Store results back in the original dataframe
rgi_df['MRBID'] = rgi_gdf['MRBID']

Let's see what we've got:

In [ ]:
rgi_df

In [ ]:
We now have a new column 

## Conclusion

